# CPW media example

Coplanar waveguide (CPW) is a planar transmission line were the return conductor is a plane located on both side of a strip conductor on the same layer. The characteristic impedance depends of the strip width and the gap with the return plane, giving two degree of freedom to designer.

A great advantage of CPW is the convenience to perform shunt or serie connexions with components, because both the signal and the return path are on the same side, thus no via are required. On the other hand, the larges return planes consume a lot of surface on the circuit.

Coplanar waveguide has quasi-TEM mode and less dispertion than microstrip. However, to avoid effect of slotline-mode, the planes on both side of the strip have to be connected together trough bond-wires. Another solution is to add a conductor plane on the backside of substrate and connect all planes together with via fences. But should the height of the sustrate be small and the gap with coplanar planes be big, then the line would behave like a microstrip line instead.

![isometric view of coplanar waveguide structure without conductor backing](coplanar_waveguide_structure.svg "coplanar waveguide structure without conductor backing")

In [ ]:
import skrf as rf
from skrf.calibration import IEEEP370_SE_NZC_2xThru
from skrf.media import CPW
import numpy as np
import matplotlib.pyplot as plt

rf.stylely()

## Measurement of two CPWG lines with different lengths

The measurement where performed the 21th March 2017 on a Anritsu MS46524B 20GHz Vector Network Analyser. The setup is a linear frequency sweep from 1MHz to 10GHz with 10'000 points. Output power is 0dBm, IF bandwidth is 1kHz and neither averaging nor smoothing are used.

CPWGxxx is a L long, W wide, with a G wide gap to top ground, T thick copper coplanar waveguide on ground on a H height substrate with top and bottom ground plane. A closely spaced via wall is placed on both side of the line and the top and bottom ground planes are connected by many vias.

| Name | L (mm) | W (mm) | G (mm) | H (mm) | T (um) | Substrate |
| :--- | ---: | ---: | ---: | ---: | ---: | :--- |
| CPW100 | 100 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |
| CPW200 | 200 | 1.70 | 0.50 | 1.55 | 50 | FR-4 |

The milling of the artwork is performed mechanically with a lateral wall of 45°.

The relative permittivity of the dielectric was assumed to be approximately 4.5 for design purpose.

![MSL100 and MSL200 illustration, both are microstripline, MSL200 is twice the length of MSL100](MSL_CPWG_100_200.jpg "MSL100 and MSL200")

## Looking at measurements

We look at measurement insertion loss phase and magnitude as well as return loss phase and magnitude.

The insertion loss of CPWG200 is twice that of CPWG100 and is mostly linear up to 6 GHz. The phase shows also a two factor. Both statements agree with the fact CPWG200 has twice the length of CPWG100.

The return loss insertion and phase have the same order of magnitude for the two networks. Insertion loss is below -20 dB at low frequency which indicate a good match of the impedance. The distance between magnitude minima is two time shorter with CPWG200 related to CPWG100, which again agrees with the physical length difference. 

In [ ]:
# Load raw measurements
TL100 = rf.Network('CPWG100.s2p')
TL200 = rf.Network('CPWG200.s2p')

# plot them all
plt.figure(figsize=(10, 10))
plt.suptitle('Raw measurements')
plt.subplot(2, 2, 1)
TL100.plot_s_db(0, 0)
TL200.plot_s_db(0, 0)
TL100.plot_s_db(1, 1)
TL200.plot_s_db(1, 1)
plt.subplot(2, 2, 2)
TL100.plot_s_deg(0, 0)
TL200.plot_s_deg(0, 0)
TL100.plot_s_deg(1, 1)
TL200.plot_s_deg(1, 1)
plt.subplot(2, 2, 3)
TL100.plot_s_db(1, 0)
TL200.plot_s_db(1, 0)
TL100.plot_s_db(0, 1)
TL200.plot_s_db(0, 1)
plt.subplot(2, 2, 4)
TL100.plot_s_deg(1, 0)
TL200.plot_s_deg(1, 0)
TL100.plot_s_deg(0, 1)
TL200.plot_s_deg(0, 1)

## Removing connectors effects

The SMA connectors cause a discontinuity in the signal path. The waves have to go trough a coaxial to planar transition with a complex three dimensionnal geometry in the launch region.

By splitting the 100 mm line into two halves and deembed them from the 200 mm line, the connector and 50 mm of line will be removed on both sides. This will remove the effects caused by the connectors.

By looking at the insertion loss phase of the sides models, we can notice that the splitting algorithm has introduced some 180° phase jumps into the insertion loss phase of the models.

In [ ]:
# deembedding using IEEEP370 impedance corrected 2xthru method
dm = IEEEP370_SE_NZC_2xThru(dummy_2xthru = TL100, name = '2xthru')
side1 = dm.s_side1
side1.name = 'side1'
side2 = dm.s_side2
side2.name = 'side2'
d_dut = dm.deembed(TL200)
d_dut.name = 'd_dut'

# plot them all
plt.figure(figsize=(10, 10))
plt.suptitle('Connector and launch model')
plt.subplot(2, 2, 1)
side1.plot_s_db(0, 0)
side2.plot_s_db(1, 1)
plt.subplot(2, 2, 2)
side1.plot_s_deg(0, 0)
side2.plot_s_deg(1, 1)
plt.subplot(2, 2, 3)
side1.plot_s_db(1, 0)
side2.plot_s_db(0, 1)
plt.subplot(2, 2, 4)
side1.plot_s_deg(1, 0)
side2.plot_s_deg(0, 1)

## Comparison of deembedded dut with CPW media results
We now compare our deembedded measurements results with simulated results from skrf CPW media.
The ep_r and tanD value are taken from "Correlating microstripline model to measurement" notebook in the same directory.

The inserstion loss agreement is very good both in phase and magnitude if we neglect the 180° jumps that were introduced by the splitting algorithm.

The return loss is more disapointing. The overall magnitude seems correct but this is the only agreement to be seen. Return losses below -20 dB are tricky to measure in general because the signal become small.

In [ ]:
ep_r = 4.421
tanD = 0.0167

cpw = CPW(frequency=d_dut.frequency, w = 1.7e-3, s = 0.5e-3, t = 50e-6, h = 1.55e-3,
        ep_r = ep_r, tand = tanD, rho = 1.7e-8, z0 = 50., has_metal_backside = True)
l = cpw.line(d = 100.0e-3, unit = 'm', embed = True, z0 = cpw.Z0)
l.name = 'model'

# plot them all
plt.figure(figsize=(10, 10))
plt.subplot(2,2,1)
d_dut.plot_s_db(0,0)
l.plot_s_db(0,0, marker = '.', linestyle = 'none', markevery = 10)
plt.subplot(2,2,2)
d_dut.plot_s_deg(0,0)
l.plot_s_deg(0,0, marker = '.', linestyle = 'none', markevery = 100)
plt.subplot(2,2,3)
d_dut.plot_s_db(1,0)
l.plot_s_db(1,0, marker = '.', linestyle = 'none', markevery = 200)
plt.subplot(2,2,4)
d_dut.plot_s_deg(1,0)
l.plot_s_deg(1,0, marker = '.', linestyle = 'none', markevery = 100)


## Checking deembedding
There is two basic check that can be done to ensure the consistency of the sides model generated by IEEEP370 2xthru algorithm.

1. Plot time step response of 2xthru, side1 and side2. Sides should looks like the two halves of 2xthru. This is the case here. Plot time step response of fixture-dut-fixture it should have the same left and right shape than 2xthru. Here we can see a small impedance mismatch which will cause a bounce in dut shape. Plot dut and see if it looks like the middle portion of fixture-dut-fixture. Here everithing seems OK outside the small bounce left and right of dut.

1. Looks at the residuals by deembedding two sides models from 2xthru. Residuals insertion magnitude shall be ± 0.1 dB and residual insertion loss phase shall be within ± 1°. This is the case for insertion loss magnitude but the phase exhibit a curious 180° step portion that is caused by the phase jumps in sides models.

In [ ]:
# compute residuals
res = side1.inv ** TL100 ** side2.inv
res.name = 'residuals'
res.s += 1e-15 # avoid numeric singularities

# extrapolate to dc for time step
TL100_dc = TL100.extrapolate_to_dc(kind='linear')
TL200_dc = TL200.extrapolate_to_dc(kind='linear')
side1_dc = side1.extrapolate_to_dc(kind='cubic')
side2_dc = side2.extrapolate_to_dc(kind='cubic')
d_dut_dc = d_dut.extrapolate_to_dc(kind='cubic')

# plot them all
# time domain
plt.figure(figsize=(8, 4))
plt.suptitle('Time domain reflexion step response (DC extrapolation)')
TL100_dc.s11.plot_z_time_step()
TL200_dc.s11.plot_z_time_step()
side1_dc.s11.plot_z_time_step()
side2_dc.s22.plot_z_time_step()
d_dut_dc.s11.plot_z_time_step()
plt.xlim(-2, 4)

# residuals frequency domain
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
res.plot_s_db(1,0)
plt.subplot(1, 2, 2)
res.plot_s_deg(1,0)